In [25]:
from setup_general import *

In [26]:
additional_name = pd.read_csv('data/typeless/additional_name.csv')
description = pd.read_csv('data/typeless/description.csv', sep=';')
dimensions = pd.read_csv('data/typeless/dimensions.csv')
events = pd.read_csv('data/typeless/events.csv')
general_info = pd.read_csv('data/typeless/general_info.csv', sep=';')
materials = pd.read_csv('data/typeless/materials.csv')
technique = pd.read_csv('data/typeless/technique.csv')

all_types = pd.read_excel('data/typeless/all_types.xlsx')


In [27]:
all_types.head()

,ID,KLASSIFIKAATOR_ID,NIMETUS
0,4824,23499,???
1,4935,32081,FDC (esimese päeva ümbrik)
2,3071,16573,ader
3,3562,17069,aer/mõla
4,3932,17444,ahi


In [28]:
train.head(1)

,full_nr,name,ks,material,commentary,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,text,class,technique,parameter,unit,value,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,collection_queue_nr,collection_additional_nr,element_count,legend,is_original,initial_info,damages,state,color,additional_text,type
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2065396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,F,97.0,NaN,1.0,NaN,NaN,NaN,NaN,hea,NaN,NaN,foto


In [29]:
additional_name = additional_name.groupby('MUSEAAL_ID', as_index=True).agg({'TEKST':' '.join})

In [30]:
additional_name = additional_name.rename(columns={'TEKST':'text'})

In [31]:
description = description.groupby('MUSEAAL_ID', as_index=True).agg({'LISATEKST':' '.join})
description = description.rename(columns={'LISATEKST':'additional_text'})

In [32]:
# for now dont use all awailable measures
dimensions = dimensions.rename(columns={'PARAMEETER':'parameter','TAIS_NR':'full_nr', 'NIMETUS':'name', 'YHIK':'unit','VAARTUS':'value'}).drop(columns=['KOMMENTAAR'])

In [33]:
dimensions = dimensions.groupby('MUSEAAL_ID', as_index=True).first()[['parameter', 'unit', 'value']]

In [34]:
events = events.groupby('MUSEAAL_ID', as_index=True).first()

In [35]:
events = events.rename(columns={'TAIS_NR':'full_nr', 'SYNDMUSE_LIIK':'event_type', 'ASUKOHT':'location', 'ALGUS':'start', 'LOPP':'end', 'ENNE_KR':'before_christ', 'RIIK_ADMIN_KOND':'country_and_unit', 'OSALEJA_ROLL':'participants_role', 'OSALEJA':'participant', 'KIHELKOND':'parish'}).drop(columns=['KUU'])

In [36]:
general_info = general_info.groupby('MUSEAAL_ID', as_index=True).first()

In [37]:
general_info = general_info.rename(columns={'ACR':'museum_abbr', 'TRT':'musealia_mark', 'TRS':'musealia_seria_nr', 'TRJ':'musealia_queue_nr', 'TRL':'musealia_additional_nr', 'KT':'collection_mark', 'KS':'ks', 'KJ':'collection_queue_nr','KL':'collection_additional_nr', 'NIMETUS':'name', 'KAHJUSTUSED':'damages', 'SEISUND':'state'}).drop(columns=['OLEMUS'])

In [38]:
materials = materials.rename(columns={'MATERJAL':'material', 'KOMMENTAAR':'commentary', 'TAIS_NR':'full_nr', 'NIMETUS':'name'})

In [39]:
materials['commentary'] = materials['commentary'].replace(np.nan,'')

In [40]:
materials = materials.groupby('MUSEAAL_ID', as_index=True).agg({'material':'>'.join, 'commentary':' '.join})

In [41]:
materials['commentary'] = materials['commentary'].replace('', np.nan)

In [42]:
technique = technique.rename(columns={'TEHNIKA':'technique','TAIS_NR':'full_nr', 'NIMETUS':'name'}).drop(columns=['KOMMENTAAR'])

In [43]:
technique = technique.groupby('MUSEAAL_ID', as_index=True).agg({'technique':'>'.join})

In [44]:
dfs = [additional_name, description, events, general_info, materials, technique, dimensions]

from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['MUSEAAL_ID'],
                                            how='outer'), dfs)

In [53]:
df_merged.index.names=['id']

In [55]:
typeless_samples = df_merged.sample(n=1000, random_state=42)

In [56]:
text = get_text_df(typeless_samples.copy())

/home/till/projects/uni/data science/museum_item_classification/prep_helpers.py:201: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text.text_features = text.text_features.apply(lambda x: x.strip())


In [57]:
text.to_csv('data/typeless/text.csv', index=True)

In [58]:
typeless_samples.to_csv('data/typeless/data.csv', index=True)
# running it through preparation.ipynb